# AgentCore Gateway Interceptor - Header Propagation Solution

## Overview

Many enterprises face a critical security challenge when building AI agent systems that access downstream APIs through MCP servers. The core problem is secure token exchange and identity propagation in multi-hop workflows where agents call tools that subsequently call downstream APIs.

Enterprises need to exchange tokens to send scoped down tokens and, least-privilege credentials to downstream APIs from MCP servers. This requires extracting principal identity and metadata from inbound JWT tokens, performing fine-grained access control based on caller credentials, and dynamically exchanging tokens to obtain appropriately scoped credentials for specific downstream API calls.
Organizations cannot simply forward the original authorization token to downstream services. If downstream services are compromised, the original token could be stolen and misused. Downstream services might have broader permissions than intended for specific operations, creating privilege escalation risks. The Confused Deputy problem emerges when services act on behalf of users without proper context validation, and tracking which service accessed what resource on behalf of which user becomes difficult without a proper audit trail. The problem intensifies in multi-tenant MCP server environments where different users from different tenants or organizations access the same AgentCore Gateway. These environments require strict data separation between tenants, each with different downstream API access patterns and permissions. The act-on-behalf pattern must be enforced consistently across all tenants.

Without a mechanism to pass inbound tokens to an intermediary layer, enterprises cannot validate token authenticity, apply fine-grained access control policies before token exchange, generate appropriately scoped tokens for specific downstream API calls, maintain proper execution context throughout multi-hop workflows, or ensure tenant isolation in shared infrastructure.

AgentCore Gateway addresses this through its Gateway interceptor and AWS Lambda target. Both AWS Lambda target integration and Gateway interceptor, serves as a layer of abstraction to perform token exchange before calling downstream APIs.  In this example, we show how you can  pass the inbound token to the AWS Lambda target function via custom headers using Gateway interceptor, perform validity checks including token signature verification and expiration checks, apply fine-grained access control based on the extracted principal, exchange the token for a narrowly scoped token specific to the downstream API, and call downstream APIs with the newly exchanged, least-privilege token.

This solution enables a secure act-on-behalf pattern where each hop gets separate scoped tokens, JWT-based execution context propagation that maintains user identity throughout the workflow, tenant isolation for proper data separation, clear audit trails for compliance, and decoupled security that keeps MCP schema intact while handling authentication separately

### Tutorial Details


| Information          | Details                                                   |
|:---------------------|:----------------------------------------------------------|
| Tutorial type        | Interactive                                               |
| AgentCore components | AgentCore Gateway                                         |
| Gateway Target type  | AWS Lambda                                                |
| Inbound Auth         | OAuth                                                     |
| Outbound Auth        | AWS IAM                                                   |
| Tutorial components  | Creating AgentCore Gateway and Invoking AgentCore Gateway |
| Tutorial vertical    | Cross-vertical                                            |
| Example complexity   | Easy                                                      |
| SDK used             | boto3                                                     |

In the first part of the tutorial we will create AmazonCore Gateway targets for Lambda

### Tutorial Architecture

![Architecture Diagram](images/04-gateway-interceptor-header-propagation.png)

---

1. **Client**: Initiates requests to Lambda-based MCP servers with authentication tokens
2. **AgentCore Gateway**: Routes client requests through interceptor for processing
3. **Gateway Interceptor**: Performs token exchange and metadata handling for client workflows
4. **Target Lambda (MCP Server)**: Receives processed requests with exchanged credentials


## Step 1: Install Dependencies

This step installs the required boto3 packages needed to interact with AWS services for creating MCP gateways.

In [ ]:
import subprocess
import sys

# Install required packages
subprocess.check_call([sys.executable, "-m", "pip", "install", "boto3", "requests"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "strands-agents"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "setuptools", "pip"])

print("✅ Dependencies installed")

## Step 2: Initialize AWS Clients and Configuration

This step sets up the necessary AWS service clients and creates a unique timestamp for resource naming to avoid conflicts.

In [ ]:
import boto3
import json
import time
import zipfile
import io
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
lambda_client = boto3.client('lambda', region_name='us-east-1')
iam_client = boto3.client('iam', region_name='us-east-1')
agentcore_client = boto3.client('bedrock-agentcore-control', region_name='us-east-1')

## Step 3: Create AgentCore Gateway Interceptor Function

This step creates the Agentcore Gateway Interceptor function that:
- Creates IAM Role: Sets up proper permissions for Agentcore Gateway Interceptor execution
- Implements Header Logic: Moves Authorization header from request headers to request body
- Handles MCP Protocol: Processes MCP gateway requests and returns transformed responses

The Agentcore Gateway Interceptor function extracts the Authorization header and places it in the request body's arguments, making it available to downstream processing.

In [ ]:
def create_interceptor_lambda_complete():
    """Create complete Agentcore Gateway Interceptor with IAM role and function"""
    lambda_client = boto3.client('lambda', region_name='us-east-1')
    iam_client = boto3.client('iam', region_name='us-east-1')
    
    # Create IAM role for Lambda
    lambda_trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    lambda_role_name = f'InterceptorLambdaRole-{timestamp}'
    
    lambda_role_response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(lambda_trust_policy),
        Description='IAM role for Interceptor Lambda'
    )
    
    # Attach basic lambda execution policy
    iam_client.attach_role_policy(
        RoleName=lambda_role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    
    lambda_role_arn = lambda_role_response['Role']['Arn']
    print(f"Lambda IAM role created: {lambda_role_arn}")
    
    # Wait for role to be available
    print("Waiting for Lambda role to be available...")
    time.sleep(10)
    
    # Lambda function code that processes authorization token for exchange
    lambda_code = '''
import json
import uuid

def lambda_handler(event, context):
    # Extract the gateway request from the correct structure
    mcp_data = event.get('mcp', {})
    gateway_request = mcp_data.get('gatewayRequest', {})
    headers = gateway_request.get('headers', {})
    body = gateway_request.get('body', {})
    extended_body = body
    
    # Extract authorization token for token exchange
    # NOTE: This authorization token is NOT meant for propagation - it is for token exchange
    auth_header = headers.get('authorization', '') or headers.get('Authorization', '')
    
    # Extract custom header for propagation
    custom_header = headers.get('customHeaderKey', '')
    
    # Code for token exchange using the authorization token would go here
    # The token received is used for exchange purposes, not for impersonation
    
    if "params" in extended_body and "arguments" in extended_body["params"]:
        # Add exchanged token or credentials to arguments (not the original auth token)
        extended_body["params"]["arguments"]["exchanged_credentials"] = "exchanged_token_placeholder"
        # Add custom header to arguments for downstream processing
        extended_body["params"]["arguments"]["customHeaderKey"] = custom_header
    
    # Return transformed request without passing the original authorization header
    response = {
        "interceptorOutputVersion": "1.0",
        "mcp": {
            "transformedGatewayRequest": {
                "headers": {
                    "Accept": "application/json",
                    "Content-Type": "application/json"
                },
                "body": extended_body
            }
        }
    }
    return response
'''
    
    # Create ZIP file for Lambda
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.writestr('lambda_function.py', lambda_code)
    
    zip_buffer.seek(0)
    
    # Create Lambda function
    lambda_function_name = f'interceptor-lambda-{timestamp}'
    
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_function_name,
        Runtime='python3.13',
        Role=lambda_role_arn,
        Handler='lambda_function.lambda_handler',
        Code={
            'ZipFile': zip_buffer.read()
        },
        Description='Interceptor Lambda for MCP Gateway'
    )
    
    lambda_arn = lambda_response['FunctionArn']
    print(f"Lambda function created: {lambda_arn}")
    
    return lambda_arn

# Create Lambda function
lambda_arn = create_interceptor_lambda_complete()
print(f"\n✅ Lambda creation completed: {lambda_arn}")

## Step 4: Create MCP Gateway with Interceptor Configuration

This step creates the MCP Gateway that:
- Creates Gateway IAM Role: Sets up permissions for the gateway service
- Configures Interceptor: Links the Lambda function as a request interceptor
- Sets Up Authentication: Configures JWT-based authentication
- Waits for Ready State: Polls until the gateway is fully operational

The gateway intercepts all incoming requests and routes them through the Lambda function before forwarding to targets.

In [ ]:
# Gateway creation
iam_response = iam_client.create_role(
    RoleName=f'BedrockAgentCoreGatewayRole-{timestamp}',
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {"Effect": "Allow", "Principal": {"Service": "bedrock-agentcore.amazonaws.com"}, "Action": "sts:AssumeRole"}
        ]
    })
)

iam_client.put_role_policy(
    RoleName=f'BedrockAgentCoreGatewayRole-{timestamp}',
    PolicyName='LambdaInvokePolicy',
    PolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "lambda:InvokeAsync",
                    "lambda:InvokeFunction"
                ],
                "Resource": "*"
            }
        ]
    })
)

gateway_response = agentcore_client.create_gateway(
    name=f"interceptor-lambda-gateway-{timestamp}",
    protocolType="MCP",
    protocolConfiguration={"mcp": {"supportedVersions": ["2025-03-26", "2025-06-18"]}},
    interceptorConfigurations=[{
        "interceptor": {"lambda": {"arn": lambda_arn}},
        "interceptionPoints": ["REQUEST"],
        "inputConfiguration": {"passRequestHeaders": True}
    }],
    authorizerType="CUSTOM_JWT",
    authorizerConfiguration={
        "customJWTAuthorizer": {
            "discoveryUrl": "DISCOVERY_URL",
            "allowedClients": ["CLIENT_ID"]
        }
    },
    roleArn=iam_response['Role']['Arn']
)

gateway_id = gateway_response['gatewayId']

# Verify gateway was created
gateway_info = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)

while True:
    status_response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    if status_response.get('status') == 'READY':
        gateway_url = status_response.get('gatewayUrl')
        break
    time.sleep(10)
print(f"Gateway created with status: {gateway_info.get('status')}")
print(f"✅ Gateway: {gateway_url}")

## Step 5: Verify Gateway Status

This step performs an additional verification check to ensure the gateway is properly configured and ready to handle requests.

In [ ]:
# Get gateway again
gateway_info_check = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
print(f"Gateway check: {gateway_info_check}")

## Step 6: Create Target Lambda Function and Gateway Target

This step creates the target Lambda function that:
- Creates Target Lambda: A simple Lambda that receives processed requests
- Sets Up IAM Role: Proper permissions for the target Lambda
- Creates Gateway Target: Registers the Lambda as an MCP target with tool schema
- Defines Tool Schema: Specifies the MCP tool interface with required parameters

The target Lambda receives requests that have been processed by the interceptor, with Authorization headers moved to the request body.

In [ ]:
def create_gateway_lambda_target_complete(gateway_id, gateway_url):
    """Create gateway Lambda target and wait for it to be ready"""
    lambda_client = boto3.client('lambda', region_name='us-east-1')
    iam_client = boto3.client('iam', region_name='us-east-1')
    
    # Create simple Lambda function for target
    target_lambda_code = '''import json

def lambda_handler(event, context):
    # Code for processing exchanged credentials would go here
    # The received request contains exchanged tokens, not the original authorization token
    
    # Extract custom header value directly from event
    custom_header_value = event.get("customHeaderKey", "")
    
    # Log only the custom header value
    print(f"Custom Header Value: {custom_header_value}")
    
    # Simple response for MCP demo tool
    response = {
        "jsonrpc": "2.0",
        "id": event.get("id", 1),
        "result": {
            "message": "Hello from Lambda target!",
            "customHeaderKey": custom_header_value
        }
    }
    
    return response
'''
    
    # Create IAM role for target Lambda
    target_lambda_trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    target_lambda_role_name = f'TargetLambdaRole-{timestamp}'
    
    target_lambda_role_response = iam_client.create_role(
        RoleName=target_lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(target_lambda_trust_policy),
        Description='IAM role for Target Lambda'
    )
    
    # Attach basic lambda execution policy
    iam_client.attach_role_policy(
        RoleName=target_lambda_role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    
    target_lambda_role_arn = target_lambda_role_response['Role']['Arn']
    print(f"Target Lambda IAM role created: {target_lambda_role_arn}")
    
    # Wait for role to be available
    print("Waiting for target Lambda role to be available...")
    time.sleep(10)
    
    # Create ZIP file for target Lambda
    target_zip_buffer = io.BytesIO()
    with zipfile.ZipFile(target_zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.writestr('lambda_function.py', target_lambda_code)
    
    target_zip_buffer.seek(0)
    
    # Create target Lambda function
    target_lambda_function_name = f'target-lambda-{timestamp}'
    
    target_lambda_response = lambda_client.create_function(
        FunctionName=target_lambda_function_name,
        Runtime='python3.13',
        Role=target_lambda_role_arn,
        Handler='lambda_function.lambda_handler',
        Code={
            'ZipFile': target_zip_buffer.read()
        },
        Description='Target Lambda for MCP Gateway'
    )
    
    target_lambda_arn = target_lambda_response['FunctionArn']
    print(f"Target Lambda function created: {target_lambda_arn}")
    target_name = f"lambda-target-{timestamp}"
    
    target_response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": target_lambda_response['FunctionArn'],
                    "toolSchema": {
                        "inlinePayload": [{
                            "description": "Tool to execute demo MCP",
                            "inputSchema": {
                                "properties": {
                                    "key1": {"description": "key1 for demo target", "type": "string"},
                                    "key2": {"description": "key2 for demo target", "type": "string"},
                                    "key3": {"description": "key3 for demo target", "type": "string"}
                                },
                                "required": ["key1"],
                                "type": "object"
                            },
                            "name": "mcp_demo"
                        }]
                    }
                }
            }
        },
        credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
    )
    
    target_id = target_response['targetId']
    
    while True:
        status_response = agentcore_client.get_gateway_target(gatewayIdentifier=gateway_id, targetId=target_id)
        if status_response.get('status') == 'READY':
            break
        time.sleep(10)
    
    print(f"✅ Target: {target_id}")
    
    return target_id, target_lambda_arn, target_name

# Create Gateway Lambda target
lambda_target_id, target_lambda_arn, target_name = create_gateway_lambda_target_complete(gateway_id, gateway_url)
print(f"\n✅ Lambda target creation completed: {lambda_target_id}")

## Step 7: Summary and Resource Information

This step displays a summary of all created resources including:
- Timestamp: Unique identifier for this deployment
- Lambda ARN: AgentCore Gateway Interceptor function
- Gateway ID: The MCP Gateway identifier
- Gateway URL: The endpoint for MCP client connections
- Target ID: The target Lambda configuration

Use the Gateway URL to connect MCP clients and test the header propagation functionality.

In [ ]:
print(f"\n🎉 COMPLETE")
print(f"📅 Timestamp: {timestamp}")
print(f"🔧 Lambda: {lambda_arn}")
print(f"🚪 Gateway: {gateway_id}")
print(f"🌐 URL: {gateway_url}")
print(f"🎯 Target: {lambda_target_id}")

## Step 8: Test MCP Gateway with Cognito Authentication

This step demonstrates how to:
- Get Cognito Token: Authenticate with the Cognito endpoint used in the gateway configuration
- Call MCP Endpoint: Make an authenticated request to the gateway using the bearer token
- Test Header Propagation: Verify that AgentCore Gateway Interceptor properly processes the Authorization header

The request uses the MCP protocol format and calls the `mcp_demo` tool defined in the target schema.

In [ ]:
import requests
import base64

# Cognito configuration
client_id = "CLIENT_ID"  # From gateway auth config
client_secret = "CLIENT_SECRET"  # Replace with actual secret
token_endpoint = "TOKEN_ENDPOINT"

# Get Cognito token
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

token_response = requests.post(
    token_endpoint,
    headers={
        "Authorization": f"Basic {auth_header}",
        "Content-Type": "application/x-www-form-urlencoded"
    },
    data="grant_type=client_credentials"
)

if token_response.status_code == 200:
    access_token = token_response.json()["access_token"]
    print(f"✅ Token obtained: {access_token[:20]}...")
    
    # List available tools first
    list_tools_body = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/list"
    }
    
    list_response = requests.post(
        gateway_url,
        headers={
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
            "customHeaderKey": "custom-metadata-value"
        },
        json=list_tools_body
    )
    
    print(f"\n📋 List Tools Response Status: {list_response.status_code}")
    print(f"📋 Available Tools: {list_response.text}")
    
    # Extract tool name from list response
    tool_name = "mcp_demo"  # default fallback
    if list_response.status_code == 200:
        try:
            tools_data = list_response.json()
            if "result" in tools_data and "tools" in tools_data["result"]:
                tools = tools_data["result"]["tools"]
                if tools and len(tools) > 0:
                    tool_name = tools[0]["name"]
                    print(f"📋 Using tool: {tool_name}")
        except Exception as e:
            print(f"⚠️ Could not parse tools list, using default: {e}")
    
    # MCP request body
    mcp_body = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": {
                "key1": "test_value_1",
                "key2": "test_value_2",
                "key3": "test_value_3"
            }
        }
    }
    
    # Call MCP gateway
    mcp_response = requests.post(
        gateway_url,
        headers={
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
            "customHeaderKey": "custom-metadata-value"
        },
        json=mcp_body
    )
    
    print(f"\n📡 MCP Response Status: {mcp_response.status_code}")
    print(f"📄 MCP Response: {mcp_response.text}")
    
else:
    print(f"❌ Token request failed: {token_response.status_code}")
    print(f"Error: {token_response.text}")

## Step 9: Using with Strands Agent

Now let's integrate with a Strands agent that supports authentication.

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

def create_streamable_http_transport():
    """Create transport with OAuth token"""
    return streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"}
    )

client = MCPClient(create_streamable_http_transport)

# Create Bedrock model
model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

print("✅ Strands agent configured with authentication")

In [ ]:
with client:
    # List available tools
    tools = client.list_tools_sync()
    
    # Create agent
    agent = Agent(model=model, tools=tools)
    
    print(f"Tools loaded: {agent.tool_names}\n")
    
    # Test 1: List tools
    print("Test 1: List available tools")
    print("=" * 50)
    response = agent("Hi, can you list all tools available to you?")
    print(f"Agent response: {response}\n")

In [ ]:
with client:
    # List available tools
    tools = client.list_tools_sync()
    
    # Create agent
    agent = Agent(model=model, tools=tools)
    
    print(f"Tools loaded: {agent.tool_names}\n")
    
    # Test 3: Direct tool call
    print("Test: Direct tool call")
    print("=" * 50)
    result = client.call_tool_sync(
        tool_use_id="get-order-123",
        name=f"{target_name}___mcp_demo",
        arguments={"key1": "test"}
    )
    print(f"Tool result: {result['content'][0]['text']}")